# Neural Model in Keras

In [1]:
import os
import sys
import pickle as pkl

import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, Reshape, LSTM, Dense, Dropout
from keras.utils import multi_gpu_model, Sequence
from keras import optimizers

import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix

sys.path.append(os.path.abspath('../preprocessing/'))
from data_parser import DataParser
from config import Config

Using TensorFlow backend.


## Data preparation

### Load and balance data

In [2]:
dp = DataParser(Config.embed_dim, Config.max_seq_len)
sequences, labels, embed_matrix = dp.load_data()
print 'parse_data', labels[:400]

sequences, labels = dp.shuffle_data(sequences, labels)
print 'shuffle', labels[:400]

x_train_seq, y_train = dp.balance_labels(sequences, labels)
print 'balance', y_train[:400]

x_train_seq, y_train = dp.shuffle_data(x_train_seq, y_train)
print 'shuffle', y_train[:400]

x_train_syn = np.random.randint(0, high=10, size=(int(x_train_seq.shape[0]), int( x_train_seq.shape[1])))
x_train_lex = np.random.rand(int(x_train_seq.shape[0]), Config.lex_feats,)

parse_data [[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
shuffle [[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
balance [[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
shuffle [[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]


### Define Model

In [3]:
seq_input = Input(shape=(Config.max_seq_len,), dtype='int32', name='seq_input')
syn_input = Input(shape=(Config.max_seq_len,), dtype='float32', name='syn_input')
lex_input = Input(shape=(Config.lex_feats,), dtype='float32', name='lex_input')

seq_embeds = Embedding(embed_matrix.shape[0], Config.embed_dim, 
                       weights=[embed_matrix], input_length=Config.max_seq_len,
                       trainable=False,
                       embeddings_regularizer=keras.regularizers.l2(.01),
                       name='embedding_layer')(seq_input)

syn_addons = Reshape((Config.max_seq_len, 1), name='reshape_layer')(syn_input)

time_series = seq_embeds
# time_series = Concatenate()([seq_embeds, syn_addons])

lstm_output = LSTM(Config.lstm_units, 
                   activation='relu',
                   name='lstm_layer')(time_series)

# concat = Concatenate()([lstm_output, lex_input])
concat = lstm_output

feed = concat
for i in range(3):
    feed = Dense(Config.lstm_units // 2,
                 activation='relu',
                 name='dense_layer_{}'.format(i))(feed)
#     feed = Dropout(.01)(feed)

predictions = Dense(Config.num_classes,
                    activation='softmax',
                    name='predictions')(feed)

model = Model(inputs=[seq_input, syn_input, lex_input], outputs=predictions)

sgd = optimizers.SGD(lr=Config.learning_rate, decay=Config.learning_decay)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
seq_input (InputLayer)       (None, 200)               0         
_________________________________________________________________
embedding_layer (Embedding)  (None, 200, 50)           808750    
_________________________________________________________________
lstm_layer (LSTM)            (None, 10)                2440      
_________________________________________________________________
dense_layer_0 (Dense)        (None, 5)                 55        
_________________________________________________________________
dense_layer_1 (Dense)        (None, 5)                 30        
_________________________________________________________________
dense_layer_2 (Dense)        (None, 5)                 30        
_________________________________________________________________
predictions (Dense)          (None, 5)                 30        
Total para

### Run model

In [ ]:
model.fit([x_train_seq, x_train_syn, x_train_lex], y_train, 
          batch_size=Config.batch_size, 
          epochs=Config.epochs, 
          validation_split=0.25)

y_pred_test = model.predict_on_batch([x_train_seq, x_train_syn, x_train_lex])
print confusion_matrix(y_train.argmax(axis=-1), y_pred_test.argmax(axis=-1))

Train on 1500 samples, validate on 500 samples
Epoch 1/50
1500/1500 [==============================] - 14s 9ms/step - loss: 3081.2282 - acc: 0.2033 - val_loss: 3081.2239 - val_acc: 0.1880
Epoch 2/50
1500/1500 [==============================] - 13s 9ms/step - loss: 3081.2227 - acc: 0.2033 - val_loss: 3081.2225 - val_acc: 0.1880
Epoch 3/50
1500/1500 [==============================] - 13s 9ms/step - loss: 3081.2220 - acc: 0.2040 - val_loss: 3081.2220 - val_acc: 0.1980
Epoch 4/50
1500/1500 [==============================] - 13s 9ms/step - loss: 3081.2215 - acc: 0.2153 - val_loss: 3081.2216 - val_acc: 0.2120
Epoch 5/50
1500/1500 [==============================] - 13s 9ms/step - loss: 3081.2214 - acc: 0.2187 - val_loss: 3081.2218 - val_acc: 0.2380
Epoch 6/50
1500/1500 [==============================] - 13s 9ms/step - loss: 3081.2212 - acc: 0.2220 - val_loss: 3081.2216 - val_acc: 0.2400
Epoch 7/50
1500/1500 [==============================] - 13s 9ms/step - loss: 3081.2212 - acc: 0.2087 - val_